In [25]:
import pyspark
from pyspark import SparkContext

##############
#### EJ 1 ####
##############

sc = SparkContext.getOrCreate() 


data = sc.textFile("data/calidad_aire_datos_meteo_mes.csv") 
header = data.first() 


rows = data.filter(lambda line: line != header) \
  .map(lambda line: line.split(";")) 


rows_madrid = rows.filter(lambda f: f[0] == "28")


rows_temp = rows_madrid.filter(lambda f: f[3] == "83") 


resultado_ej1 = rows_temp.filter(
    lambda f: any(f[k] == "V" for k in range(9, min(57, len(f)), 2))
).count() 
print("")
print("############ Ejercicio 1 ############")
print("")

print("Nº de registros de temperatura con >=1 hora válida:", resultado_ej1)

##############
#### EJ 2 ####
##############


rows_temp = rows.filter(lambda f: f[0] == "28" and f[3] == "83")


resultado_ej2 = (
    rows_temp
    
    .flatMap(
        lambda c: [
            (
                f"{c[5]}-{c[6].zfill(2)}-{c[7].zfill(2)}", 
                float(c[8 + 2 * i].replace(',', '.'))      
            )
            for i in range(24) if c[9 + 2 * i] == "V"     
        ]
    )
    
    .reduceByKey(lambda a, b: a if a > b else b)
    
    
    .sortByKey()
)

temperaturas_maximas = resultado_ej2.collect()
print("")
print("")
print("############ Ejercicio 2 ############")
print("")

for fecha, t_max in temperaturas_maximas:
    print(f"Fecha: {fecha} | Temp Máxima: {t_max} °C")


##############
#### EJ 3 ####
##############

# M(r): Filtrado por provincia y magnitud pluviométrica (89)
rows_precip = rows.filter(lambda f: f[0] == "28" and f[3] == "89") # [cite: 105, 282]

# W: Transformación integral (map) para proyectar τ(r) y (σ(r), Σ(r))
# La comprensión de listas interna resuelve la sumatoria Σ aplicando la indicatriz de validez
precipitaciones_diarias = rows_precip.map(
    lambda c: (
        f"{c[5]}-{c[6].zfill(2)}-{c[7].zfill(2)}", # Clave τ(r): Fecha [cite: 283]
        (
            c[1], # Municipio [cite: 283]
            c[2], # Estación [cite: 283]
            sum([float(c[8 + 2 * i].replace(',', '.')) for i in range(24) if c[9 + 2 * i] == "V"]) # Σ(r) 
        )
    )
)

# S(τ): Reducción por clave temporal para hallar el supremo local (reduceByKey)
maximos_diarios_estacion = (
    precipitaciones_diarias
    .reduceByKey(lambda a, b: a if a[2] > b[2] else b) # [cite: 140, 142]
    .sortByKey() # [cite: 147, 149]
)

# Ω: Reducción escalar global (reduce) para extraer el supremo absoluto 
# Compara pares completos (fecha, (municipio, estacion, precipitación)) y conserva el de mayor magnitud
maximo_absoluto = maximos_diarios_estacion.reduce(
    lambda a, b: a if a[1][2] > b[1][2] else b # [cite: 145]
)

# Extracción de la topología local (collect) al driver [cite: 155, 157]
resultados_locales = maximos_diarios_estacion.collect() # [cite: 155, 157]
print("")
print("")
print("############ Ejercicio 3 ############")
print("")
print("--- MÁXIMOS DIARIOS POR ESTACIÓN ---")
for fecha, (municipio, estacion, total_precip) in resultados_locales:
    print(f"Fecha: {fecha} | Municipio: {municipio} | Estación: {estacion} | Precipitación Total: {total_precip}") # [cite: 283]

print("\n--- MÁXIMO ABSOLUTO DEL PERIODO ---")
print(f"Fecha: {maximo_absoluto[0]} | Municipio: {maximo_absoluto[1][0]} | Estación: {maximo_absoluto[1][1]} | Precipitación: {maximo_absoluto[1][2]}") # [cite: 284]


##############
#### EJ 4 ####
##############

# M(r): Filtrado por provincia y magnitud de temperatura (83)
rows_temp = rows.filter(lambda f: f[0] == "28" and f[3] == "83")

# Cálculo previo de μ(r): Extraemos municipio, estación, fecha y la media diaria
# Utilizamos un filter intermedio para evitar divisiones por cero si un día no tiene horas válidas
estaciones_medias = (
    rows_temp.map(
        lambda c: (
            c[1], # Municipio [cite: 105]
            c[2], # Estación [cite: 105]
            f"{c[5]}-{c[6].zfill(2)}-{c[7].zfill(2)}", # Clave temporal τ(r) [cite: 105]
            # Extraemos V_r (lista de valores de temperatura válidos)
            [float(c[8 + 2 * i].replace(',', '.')) for i in range(24) if c[9 + 2 * i] == "V"] 
        )
    )
    .filter(lambda x: len(x[3]) > 0) # Aseguramos |V_r| > 0 [cite: 279]
    .map(lambda x: (x[0], x[1], x[2], sum(x[3]) / len(x[3]))) # Transformación a media μ(r)
)

# E_A: Proyección del subconjunto de referencia (Municipio 6, Estación 4) a formato (Clave, Valor)
estacion_A = (
    estaciones_medias
    .filter(lambda x: x[0] == "6" and x[1] == "4")
    .map(lambda x: (x[2], x[3])) # Genera (fecha, media_A)
)

# E_B: Proyección del subconjunto comparado (Municipio 5, Estación 2) a formato (Clave, Valor)
estacion_B = (
    estaciones_medias
    .filter(lambda x: x[0] == "5" and x[1] == "2") 
    .map(lambda x: (x[2], x[3])) # Genera (fecha, media_B)
)

# J y ρ: Producto relacional (join) y funcional de porcentaje porcentual
comparacion_porcentual = (
    estacion_A.join(estacion_B) # Unimos por fecha. Resultado: (fecha, (media_A, media_B)) [cite: 152, 156]
    .map(lambda j: (j[0], (j[1][1] / j[1][0]) * 100)) # Aplicamos ρ(τ) = (media_B / media_A) * 100 
    .sortByKey() # Orden cronológico [cite: 147]
)

# Despliegue de resultados en el driver
resultados_ej4 = comparacion_porcentual.collect() 
print("")
print("")
print("############ Ejercicio 4 ############")
print("")
print("--- COMPARACIÓN PORCENTUAL DE TEMPERATURAS MEDIAS ---")
for fecha, porcentaje in resultados_ej4:
    print(f"Fecha: {fecha} | Porcentaje (Est. 5-2 vs Ref 6-4): {porcentaje:.2f}%") 


############ Ejercicio 1 ############

Nº de registros de temperatura con >=1 hora válida: 224


############ Ejercicio 2 ############

Fecha: 2026-02-01 | Temp Máxima: 11.7 °C
Fecha: 2026-02-02 | Temp Máxima: 12.2 °C
Fecha: 2026-02-03 | Temp Máxima: 9.8 °C
Fecha: 2026-02-04 | Temp Máxima: 11.3 °C
Fecha: 2026-02-05 | Temp Máxima: 15.5 °C
Fecha: 2026-02-06 | Temp Máxima: 10.3 °C
Fecha: 2026-02-07 | Temp Máxima: 9.0 °C
Fecha: 2026-02-08 | Temp Máxima: 12.7 °C


############ Ejercicio 3 ############

--- MÁXIMOS DIARIOS POR ESTACIÓN ---
Fecha: 2026-02-01 | Municipio: 120 | Estación: 1 | Precipitación Total: 20.8
Fecha: 2026-02-02 | Municipio: 161 | Estación: 1 | Precipitación Total: 18.4
Fecha: 2026-02-03 | Municipio: 127 | Estación: 4 | Precipitación Total: 7.6
Fecha: 2026-02-04 | Municipio: 45 | Estación: 2 | Precipitación Total: 11.6
Fecha: 2026-02-05 | Municipio: 115 | Estación: 3 | Precipitación Total: 30.8
Fecha: 2026-02-06 | Municipio: 67 | Estación: 1 | Precipitación Total: 6.3
F